# **Deep learning psoriasis/rosacea**


## Torch, torchvision, fastai

In [ ]:
!pip install "torch==1.4" "torchvision==0.5.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.4 (from versions: 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.4


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!python --version

Python 3.9.16


In [ ]:
!pip install fastai

In [ ]:
!pip list -v | grep fastai

fastai                        2.7.12               /usr/local/lib/python3.9/dist-packages pip


## Libraries and images upload

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Images_folder"
path

'/content/drive/MyDrive/Colab Notebooks/tfm_bioinformatica/Dataset2/Imagenes'

In [ ]:
import fastai
from fastai.basics import *
from fastai.vision.all import *
from fastai.metrics import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.model_selection import LeaveOneOut

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Models

### CNN

In [ ]:
# Creamos el modelo
model_cnn = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(64*28*28, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)

# Crea un objeto LeaveOneOut
loo = LeaveOneOut()

# Crear listas vacías para cada métrica
accuracy_list1 = []
precision_list1 = []
recall_list1 = []
f1score_list1 = []

# Crea listas para almacenar las predicciones y las etiquetas verdaderas
all_preds1 = []
all_labels1 = []

# Itera sobre cada conjunto de entrenamiento y prueba en el esquema LOO
for train_idx, test_idx in loo.split(get_image_files(path)):
    # Obtiene los índices de los elementos de entrenamiento y prueba
    train_items, test_items = get_image_files(path)[train_idx], get_image_files(path)[test_idx]

    # Crea un objeto DataBlock para cargar las imágenes de entrenamiento y prueba
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=aug_transforms())

    # Crea un objeto DataLoader para los datos de entrenamiento y prueba
    dls = dblock.dataloaders(path, bs=16, train=train_items, valid=test_items)

    # Crear una nueva red neuronal convolucional con arquitectura ResNet-18 para este pliegue
    learn_cnn = Learner(dls, model_cnn)

    # Entrena el modelo
    learn_cnn.fine_tune(5)

    # Realiza la evaluación en los datos de prueba
    preds, labels = learn_cnn.get_preds(dl=dls.valid)
    all_preds1.extend(torch.argmax(preds, dim=1).tolist())
    all_labels1.extend(labels.tolist())

    # Calcula las métricas utilizando sklearn.metrics
    accuracy_list1.append(accuracy_score(labels, torch.argmax(preds, dim=1)))
    precision_list1.append(precision_score(labels, torch.argmax(preds, dim=1), average='macro'))
    recall_list1.append(recall_score(labels, torch.argmax(preds, dim=1), average='macro'))
    f1score_list1.append(f1_score(labels, torch.argmax(preds, dim=1), average='macro'))

In [ ]:
import statistics

# Calcular el coeficiente de Cohen kappa global
kappa_global1 = cohen_kappa_score(all_labels1, all_preds1)

# Calcular la media y desviación estándar de cada métrica a partir de la lista correspondiente
accuracy_mean1 = statistics.mean(accuracy_list1)
accuracy_std1 = statistics.stdev(accuracy_list1)
precision_mean1 = statistics.mean(precision_list1)
precision_std1 = statistics.stdev(precision_list1)
recall_mean1 = statistics.mean(recall_list1)
recall_std1 = statistics.stdev(recall_list1)
f1score_mean1 = statistics.mean(f1score_list1)
f1score_std1 = statistics.stdev(f1score_list1)

# Imprimir los resultados en pantalla
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_mean1, accuracy_std1))
print("Precision: {:.4f} ± {:.4f}".format(precision_mean1, precision_std1))
print("Recall: {:.4f} ± {:.4f}".format(recall_mean1, recall_std1))
print("F1-score: {:.4f} ± {:.4f}".format(f1score_mean1, f1score_std1))
print("Cohen's Kappa: {:.4f}".format(kappa_global1))

In [ ]:
# Crear un diccionario con los resultados
results1 = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', "Cohen's Kappa"],
    'Mean': [accuracy_mean1, precision_mean1, recall_mean1, f1score_mean1, kappa_global1],
    'Std': [accuracy_std1, precision_std1, recall_std1, f1score_std1, 0]
}

# Crear el DataFrame
df1 = pd.DataFrame(results1)

# Guardar el DataFrame en un archivo CSV
df1.to_csv('/content/drive/MyDrive/Colab Notebooks/tfm_bioinformatica/metric_results/metric_results1.csv', index=False)

### ResNet-18

In [ ]:
# Crea un objeto LeaveOneOut
loo = LeaveOneOut()

# Crear listas vacías para cada métrica
accuracy_list2 = []
precision_list2 = []
recall_list2 = []
f1score_list2 = []

# Crea listas para almacenar las predicciones y las etiquetas verdaderas
all_preds2 = []
all_labels2 = []

# Itera sobre cada conjunto de entrenamiento y prueba en el esquema LOO
for train_idx, test_idx in loo.split(get_image_files(path)):
  # Obtiene los índices de los elementos de entrenamiento y prueba
    train_items, test_items = get_image_files(path)[train_idx], get_image_files(path)[test_idx]

    # Crea un objeto DataBlock para cargar las imágenes de entrenamiento y prueba
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=aug_transforms())

    # Crea un objeto DataLoader para los datos de entrenamiento y prueba
    dls = dblock.dataloaders(path, bs=16, train=train_items, valid=test_items)

    # Crea un modelo de aprendizaje automático
    learn_rn18 = cnn_learner(dls, resnet18, pretrained=True)

    # Entrena el modelo
    learn_rn18.fine_tune(5)

    # Realiza la evaluación en los datos de prueba
    preds, labels = learn_rn18.get_preds(dl=dls.valid)
    all_preds2.extend(torch.argmax(preds, dim=1).tolist())
    all_labels2.extend(labels.tolist())

    # Calcula las métricas utilizando sklearn.metrics
    accuracy_list2.append(accuracy_score(labels, torch.argmax(preds, dim=1)))
    precision_list2.append(precision_score(labels, torch.argmax(preds, dim=1), average='macro'))
    recall_list2.append(recall_score(labels, torch.argmax(preds, dim=1), average='macro'))
    f1score_list2.append(f1_score(labels, torch.argmax(preds, dim=1), average='macro'))

In [ ]:
import statistics

# Calcular el coeficiente de Cohen kappa global
kappa_global2 = cohen_kappa_score(all_labels2, all_preds2)

# Calcular la media y desviación estándar de cada métrica a partir de la lista correspondiente
accuracy_mean2 = statistics.mean(accuracy_list2)
accuracy_std2 = statistics.stdev(accuracy_list2)
precision_mean2 = statistics.mean(precision_list2)
precision_std2 = statistics.stdev(precision_list2)
recall_mean2 = statistics.mean(recall_list2)
recall_std2 = statistics.stdev(recall_list2)
f1score_mean2 = statistics.mean(f1score_list2)
f1score_std2 = statistics.stdev(f1score_list2)

# Imprimir los resultados en pantalla
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_mean2, accuracy_std2))
print("Precision: {:.4f} ± {:.4f}".format(precision_mean2, precision_std2))
print("Recall: {:.4f} ± {:.4f}".format(recall_mean2, recall_std2))
print("F1-score: {:.4f} ± {:.4f}".format(f1score_mean2, f1score_std2))
print("Cohen's Kappa: {:.4f}".format(kappa_global2))

In [ ]:
# Crear un diccionario con los resultados
results2 = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', "Cohen's Kappa"],
    'Mean': [accuracy_mean2, precision_mean2, recall_mean2, f1score_mean2, kappa_global2],
    'Std': [accuracy_std2, precision_std2, recall_std2, f1score_std2, 0]
    }

# Crear el DataFrame
df2 = pd.DataFrame(results2)

# Guardar el DataFrame en un archivo CSV
df2.to_csv('/content/drive/MyDrive/Colab Notebooks/tfm_bioinformatica/metric_results/metric_results2.csv', index=False)

### Vgg16

In [ ]:
# Crea un objeto LeaveOneOut
loo = LeaveOneOut()

# Crear listas vacías para cada métrica
accuracy_list3 = []
precision_list3 = []
recall_list3 = []
f1score_list3 = []

# Crea listas para almacenar las predicciones y las etiquetas verdaderas
all_preds3 = []
all_labels3 = []

# Itera sobre cada conjunto de entrenamiento y prueba en el esquema LOO
for train_idx, test_idx in loo.split(get_image_files(path)):
    # Obtiene los índices de los elementos de entrenamiento y prueba
    train_items, test_items = get_image_files(path)[train_idx], get_image_files(path)[test_idx]

    # Crea un objeto DataBlock para cargar las imágenes de entrenamiento y prueba
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=aug_transforms())

    # Crea un objeto DataLoader para los datos de entrenamiento y prueba
    dls = dblock.dataloaders(path, bs=16, train=train_items, valid=test_items)

    # Crear una red neuronal convolucional con arquitectura VGG16
    learn_vgg = cnn_learner(dls, vgg16, pretrained=True)

    # Entrena el modelo
    learn_vgg.fine_tune(5)

    # Realiza la evaluación en los datos de prueba
    preds, labels = learn_vgg.get_preds(dl=dls.valid)
    all_preds3.extend(torch.argmax(preds, dim=1).tolist())
    all_labels3.extend(labels.tolist())

    # Calcula las métricas utilizando sklearn.metrics
    accuracy_list3.append(accuracy_score(labels, torch.argmax(preds, dim=1)))
    precision_list3.append(precision_score(labels, torch.argmax(preds, dim=1), average='macro'))
    recall_list3.append(recall_score(labels, torch.argmax(preds, dim=1), average='macro'))
    f1score_list3.append(f1_score(labels, torch.argmax(preds, dim=1), average='macro'))

In [ ]:
import statistics

# Calcular el coeficiente de Cohen kappa global
kappa_global3 = cohen_kappa_score(all_labels3, all_preds3)

# Calcular la media y desviación estándar de cada métrica a partir de la lista correspondiente
accuracy_mean3 = statistics.mean(accuracy_list3)
accuracy_std3 = statistics.stdev(accuracy_list3)
precision_mean3 = statistics.mean(precision_list3)
precision_std3 = statistics.stdev(precision_list3)
recall_mean3 = statistics.mean(recall_list3)
recall_std3 = statistics.stdev(recall_list3)
f1score_mean3 = statistics.mean(f1score_list3)
f1score_std3 = statistics.stdev(f1score_list3)

# Imprimir los resultados en pantalla
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_mean3, accuracy_std3))
print("Precision: {:.4f} ± {:.4f}".format(precision_mean3, precision_std3))
print("Recall: {:.4f} ± {:.4f}".format(recall_mean3, recall_std3))
print("F1-score: {:.4f} ± {:.4f}".format(f1score_mean3, f1score_std3))
print("Cohen's Kappa: {:.4f}".format(kappa_global3))

In [ ]:
# Crear un diccionario con los resultados
results3 = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', "Cohen's Kappa"],
    'Mean': [accuracy_mean3, precision_mean3, recall_mean3, f1score_mean3, kappa_global3],
    'Std': [accuracy_std3, precision_std3, recall_std3, f1score_std3, 0]
}

# Crear el DataFrame
df3 = pd.DataFrame(results3)

# Guardar el DataFrame en un archivo CSV
df3.to_csv('/content/drive/MyDrive/Colab Notebooks/tfm_bioinformatica/metric_results/metric_results3.csv', index=False)

### Efficient-Net-V2

In [ ]:
# Importamos EfficientNetV2
!pip install timm
from timm import create_model

# Crea un objeto LeaveOneOut
loo = LeaveOneOut()

# Crear listas vacías para cada métrica
accuracy_list4 = []
precision_list4 = []
recall_list4 = []
f1score_list4 = []

# Crea listas para almacenar las predicciones y las etiquetas verdaderas
all_preds4 = []
all_labels4 = []

# Itera sobre cada conjunto de entrenamiento y prueba en el esquema LOO
for train_idx, test_idx in loo.split(get_image_files(path)):
    # Obtiene los índices de los elementos de entrenamiento y prueba
    train_items, test_items = get_image_files(path)[train_idx], get_image_files(path)[test_idx]

    # Crea un objeto DataBlock para cargar las imágenes de entrenamiento y prueba
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=aug_transforms())

    # Crea un objeto DataLoader para los datos de entrenamiento y prueba
    dls = dblock.dataloaders(path, bs=16, train=train_items, valid=test_items)

    # Crear un modelo EfficientNetV2 pre-entrenado
    model_effnetv2 = create_model('tf_efficientnetv2_s', pretrained=True)
    n_features = model_effnetv2.classifier.in_features

    # Reemplazar el clasificador lineal de la red por uno nuevo
    model_effnetv2.classifier = nn.Linear(n_features, len(dls.vocab))

    # Crear una nueva red neuronal convolucional con arquitectura ResNet-18 para este pliegue
    learn_effnetv2 = Learner(dls, model_effnetv2)

    # Entrenar el modelo en este pliegue y obtener las métricas
    learn_effnetv2.fine_tune(5)

    # Realiza la evaluación en los datos de prueba
    preds, labels = learn_effnetv2.get_preds(dl=dls.valid)
    all_preds4.extend(torch.argmax(preds, dim=1).tolist())
    all_labels4.extend(labels.tolist())

    # Calcula las métricas utilizando sklearn.metrics
    accuracy_list4.append(accuracy_score(labels, torch.argmax(preds, dim=1)))
    precision_list4.append(precision_score(labels, torch.argmax(preds, dim=1), average='macro'))
    recall_list4.append(recall_score(labels, torch.argmax(preds, dim=1), average='macro'))
    f1score_list4.append(f1_score(labels, torch.argmax(preds, dim=1), average='macro'))

In [ ]:
import statistics

# Calcular el coeficiente de Cohen kappa global
kappa_global4 = cohen_kappa_score(all_labels4, all_preds4)

# Calcular la media y desviación estándar de cada métrica a partir de la lista correspondiente
accuracy_mean4 = statistics.mean(accuracy_list4)
accuracy_std4 = statistics.stdev(accuracy_list4)
precision_mean4 = statistics.mean(precision_list4)
precision_std4 = statistics.stdev(precision_list4)
recall_mean4 = statistics.mean(recall_list4)
recall_std4 = statistics.stdev(recall_list4)
f1score_mean4 = statistics.mean(f1score_list4)
f1score_std4 = statistics.stdev(f1score_list4)

# Imprimir los resultados en pantalla
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_mean4, accuracy_std4))
print("Precision: {:.4f} ± {:.4f}".format(precision_mean4, precision_std4))
print("Recall: {:.4f} ± {:.4f}".format(recall_mean4, recall_std4))
print("F1-score: {:.4f} ± {:.4f}".format(f1score_mean4, f1score_std4))
print("Cohen's Kappa: {:.4f}".format(kappa_global4))

In [ ]:
# Crear un diccionario con los resultados
results4 = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', "Cohen's Kappa"],
    'Mean': [accuracy_mean4, precision_mean4, recall_mean4, f1score_mean4, kappa_global4],
    'Std': [accuracy_std4, precision_std4, recall_std4, f1score_std4, 0]
}

# Crear el DataFrame
df4 = pd.DataFrame(results4)

# Guardar el DataFrame en un archivo CSV
df4.to_csv('/content/drive/MyDrive/Colab Notebooks/tfm_bioinformatica/metric_results/metric_results4.csv', index=False)

### MobileNet-v2

In [ ]:
# Crea un objeto LeaveOneOut
loo = LeaveOneOut()

# Crear listas vacías para cada métrica
accuracy_list5 = []
precision_list5 = []
recall_list5 = []
f1score_list5 = []

# Crea listas para almacenar las predicciones y las etiquetas verdaderas
all_preds5 = []
all_labels5 = []

# Itera sobre cada conjunto de entrenamiento y prueba en el esquema LOO
for train_idx, test_idx in loo.split(get_image_files(path)):
    # Obtiene los índices de los elementos de entrenamiento y prueba
    train_items, test_items = get_image_files(path)[train_idx], get_image_files(path)[test_idx]

    # Crea un objeto DataBlock para cargar las imágenes de entrenamiento y prueba
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       item_tfms=Resize(224),
                       batch_tfms=aug_transforms())

    # Crea un objeto DataLoader para los datos de entrenamiento y prueba
    dls = dblock.dataloaders(path, bs=16, train=train_items, valid=test_items)

    # Crear un modelo MobileNetV2 pre-entrenado
    model_mobilenetv2 = models.mobilenet_v2(pretrained=True)
    n_features = model_mobilenetv2.classifier[1].in_features

    # Reemplazar el clasificador lineal de la red por uno nuevo
    model_mobilenetv2.classifier[1] = nn.Linear(n_features, len(dls.vocab))

    # Crear el objeto Learner
    learn_mobilenetv2 = Learner(dls, model_mobilenetv2)

    # Entrenar el modelo en este pliegue y obtener las métricas
    learn_mobilenetv2.fine_tune(5)

    # Realiza la evaluación en los datos de prueba
    preds, labels = learn_mobilenetv2.get_preds(dl=dls.valid)
    all_preds5.extend(torch.argmax(preds, dim=1).tolist())
    all_labels5.extend(labels.tolist())

    # Calcula las métricas utilizando sklearn.metrics
    accuracy_list5.append(accuracy_score(labels, torch.argmax(preds, dim=1)))
    precision_list5.append(precision_score(labels, torch.argmax(preds, dim=1), average='macro'))
    recall_list5.append(recall_score(labels, torch.argmax(preds, dim=1), average='macro'))
    f1score_list5.append(f1_score(labels, torch.argmax(preds, dim=1), average='macro'))

In [ ]:
import statistics

# Calcular el coeficiente de Cohen kappa global
kappa_global5 = cohen_kappa_score(all_labels5, all_preds5)

# Calcular la media y desviación estándar de cada métrica a partir de la lista correspondiente
accuracy_mean5 = statistics.mean(accuracy_list5)
accuracy_std5 = statistics.stdev(accuracy_list5)
precision_mean5 = statistics.mean(precision_list5)
precision_std5 = statistics.stdev(precision_list5)
recall_mean5 = statistics.mean(recall_list5)
recall_std5 = statistics.stdev(recall_list5)
f1score_mean5 = statistics.mean(f1score_list5)
f1score_std5 = statistics.stdev(f1score_list5)

# Imprimir los resultados en pantalla
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_mean5, accuracy_std5))
print("Precision: {:.4f} ± {:.4f}".format(precision_mean5, precision_std5))
print("Recall: {:.4f} ± {:.4f}".format(recall_mean5, recall_std5))
print("F1-score: {:.4f} ± {:.4f}".format(f1score_mean5, f1score_std5))
print("Cohen's Kappa: {:.4f}".format(kappa_global5))

In [ ]:
# Crear un diccionario con los resultados
results5 = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', "Cohen's Kappa"],
    'Mean': [accuracy_mean5, precision_mean5, recall_mean5, f1score_mean5, kappa_global5],
    'Std': [accuracy_std5, precision_std5, recall_std5, f1score_std5, 0]
}

# Crear el DataFrame
df5 = pd.DataFrame(results5)

# Guardar el DataFrame en un archivo CSV
df5.to_csv('/content/drive/MyDrive/Colab Notebooks/tfm_bioinformatica/metric_results/metric_results5.csv', index=False)

## Statistics analysis

In [ ]:
import pandas as pd
from scipy.stats import shapiro

# Ruta de la carpeta donde se encuentran los archivos CSV
folder_path = '/content/drive/MyDrive/Colab Notebooks/tfm_bioinformatica/metric_results/'

# Lista para almacenar los datos de los modelos
data = []
model_names = ['CNN', 'ResNet-18', 'Vgg-16', 'EfficientNet-v2', 'MobileNet-v2']

# Cargar los datos de los CSVs y mostrar los resultados
for i, model_name in enumerate(model_names):
    filename = folder_path + f"metric_results{i+1}.csv"
    df = pd.read_csv(filename)
    data.append(df['Mean'])

# Realizar el análisis de normalidad para cada grupo
normality_test_passed = True
for i in range(len(data)):
    print(f"Análisis de normalidad para el modelo {model_names[i]}:")
    stat, p = shapiro(data[i])
    print(f"Estadístico de prueba: {stat}")
    print(f"Valor p: {p}")
    if p > 0.05:
        print("La muestra sigue una distribución normal.")
    else:
        print("La muestra no sigue una distribución normal.")
        normality_test_passed = False
    print()

In [ ]:
import numpy as np
from scipy.stats import kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt

# Realizar el test de Kruskal-Wallis
stat, p = kruskal(*data)
print("Test de Kruskal-Wallis:")
print(f"Estadístico de prueba: {stat}")
print(f"Valor p: {p}")
print()

# Realizar comparaciones post hoc utilizando la prueba de Tukey HSD
if p > 0.05:
    print("No se encontraron diferencias significativas entre los grupos. No es necesario realizar comparaciones post hoc.")
else:
    print("Realizando comparaciones post hoc utilizando la prueba de Tukey HSD:")
    posthoc_results = pairwise_tukeyhsd(np.concatenate(data), np.repeat(model_names, [len(d) for d in data]))
    print(posthoc_results)

    # Representación de los resultados
    posthoc_results.plot_simultaneous()
    plt.show()